# Utilities
> Useful function and utilities

In [1]:
# default_exp utils

## Image

### Cleaning up error image

In [2]:
# export
import os
import logging
from typing import Callable, List
from pathlib import Path

from IPython.core.magic import register_cell_magic
from IPython.display import Javascript
from ipywidgets import Output, VBox, Button, HTML
from PIL import Image
from tqdm.notebook import tqdm
from uuid import uuid4

In some cases, we have error image that will interrupt model trainging. We can use ```clean_error_img``` to clean the image folder

In [3]:
# export
def check_img(
    img: Path,
    formats: List[str] = [".jpg", ".jpeg", ".png", ".bmp"],
) -> None:
    """
    Check on a single image,
    If it's quality is troublesome
        we unlink/ditch the image
    """
    img = Path(img)
    # check if this path is an image
    if img.suffix.lower().split("?")[0] not in formats:
        return

    try:
        # try to open that image
        _ = Image.open(img).load()
    except Exception as e:
        if img.exists():
            img.unlink()
            logging.warning(f"removed error img: {img}")


def clean_error_img(
    path: Path,
    progress: bool = True,
) -> None:
    """
    - path: an image directory
    - progress: do we print out progress bar or not
        default True
    """
    path = Path(path)

    # check directory existence
    if path.exists() == False:
        raise FileExistsError(
            f"""path does not exists on:{path}, 
    make sure there is a directory "{path.name}".
    under directory "{path.parent}"
    """)

    # create iterator, probably with progress bar
    iterator = tqdm(list(path.iterdir()), leave=False)\
        if progress else path.iterdir()

    for obj in iterator:
        if obj.is_dir():
            # use recursion to clean the sub folder
            clean_error_img(obj, progress=progress)
        else:
            # cheking on a single image
            check_img(obj)

## Testing

### test image cleaner

In [4]:
# hide
def _testing_image_cleaner(func: Callable) -> None:
    path = Path("./test_remove")
    os.system(f"rm -rf {path}")
    path.mkdir()
    with open(path/f"file_1.JPEG", "w") as f:
        f.write("fake image")
        
    with open(path/f"file_1.jpg", "w") as f:
        f.write("fake image")
        
    with open(path/f"file_1.txt", "w") as f:
        f.write("not image")
        
    with open(path/f"file_2.png", "w") as f:
        f.write("fake image")
    
    sub1 = path/"sub1"
    sub2 = path/"sub2"
    sub1.mkdir()
    sub2.mkdir()
    sub3 = sub2/"sub3"
    sub3.mkdir()
    
    with open(sub1/"file3.BMP", "w") as f:
        f.write("fake image")
        
    with open(sub1/"file3 haha.txt", "w") as f:
        f.write("not image")
        
    with open(sub1/"😱file3 haha.jpg", "w") as f:
        f.write("fake image")
        
    with open(sub2/"file3.jpg", "w") as f:
        f.write("fake image")
    
    with open(sub2/"file4.jpeg", "w") as f:
        f.write("fake image")
        
    file5 = sub3/"file_5.jpeg"
    with open(file5, "w") as f:
        f.write("fake image")
        
    func(path)
    for p in [path, sub1, sub2, sub3]:
        print(list(p.iterdir()))
    os.system(f"rm -rf {path}")

In [5]:
# hide
testing_image_cleaner(clean_error_img)

[WindowsPath('test_remove/file_1.txt'), WindowsPath('test_remove/sub1'), WindowsPath('test_remove/sub2')]
[WindowsPath('test_remove/sub1/file3 haha.txt')]
[WindowsPath('test_remove/sub2/sub3')]
[]


## Magic function ```hush```
> Run things quietly...

This is a magical cell function, so remember to use the ```%%```

Now we have 2 versions of hush, the 1st on is the backup one, that using ipywidget as event trigger

In [6]:
# hide
# @register_cell_magic
def hush(line, cell):
    """
    A magic cell function to collapse the print out
    %%hush
    how_loud = 100
    how_verbose = "very"
    do_some_python_thing(how_loud, how_verbose)
    """
    # the current output widget
    out = Output()
    output_box = VBox([out])
    # default setting is to hide the print out
    output_box.layout.display = "none"
    # the toggling button
    show_btn = Button(description="Show output")

    def toggle_output(o):
        # show output, change the button to hide
        if output_box.layout.display == "none":
            output_box.layout.display = "block"
            show_btn.description = "Hide output"
        # hide output, change the button to show
        else:
            output_box.layout.display = "none"
            show_btn.description = "Show output"

    # assign toggle to event
    show_btn.on_click(toggle_output)

    # A control panel containing a button
    # and the output box
    total_control = VBox([show_btn, output_box])
    display(total_control)

    with out:
        ishell = get_ipython()
        # excute the code in cell
        result = ishell.run_cell(
            cell, silent=False)

    # we still want the error to be proclaimed loudly
    if result.error_in_exec:
        logging.error(f"'{result.error_in_exec}' happened, breaking silence now")
        result.raise_error()

But this version of ```toggle action``` will be stuck by the ongoing interactive

If the process is working on some thing, you can't toggle until the end of the execution.

The further improvement will be move the toggle entirely to JavaScript, hence the second version

### Hush event in JS
> As not going through any amount of python backend after run

In [14]:
# export

@register_cell_magic
def hush(line, cell):
    """
    A magic cell function to collapse the print out
    %%hush
    how_loud = 100
    how_verbose = "very"
    do_some_python_thing(how_loud, how_verbose)
    """
    # the current output widget
    out = Output()
    output_box = VBox([out])
    
    # create uuid for DOM identifying
    uuid = str(uuid4())

    # default setting is to hide the print out
    output_box.layout.display = "none"
    # the toggling button
    show_btn = Button(description="toggle output")
    show_btn.add_class(f"hush_toggle_{uuid}")
    output_box.add_class(f"hush_output_{uuid}")
    
    
    # A control panel containing a button
    # and the output box
    total_control = VBox([show_btn, output_box])
    display(total_control)
    
    # assign JS listener
    display(Javascript(f"""
    console.info("loading toggle event: {uuid}")
    const toggle_hush = (e) =>\u007b
        var op = document.querySelector(".hush_output_{uuid}");
        if(op.style.display=="none")\u007b
             op.style.display="block"

        \u007d else \u007b
            op.style.display="none"
        \u007d

    \u007d
    document.querySelector('.hush_toggle_{uuid}').onclick=toggle_hush
    """))

    with out:
        ishell = get_ipython()
        # excute the code in cell
        result = ishell.run_cell(
            cell, silent=False)

    # we still want the error to be proclaimed loudly
    if result.error_in_exec:
        logging.error(f"'{result.error_in_exec}' happened, breaking silence now")
        result.raise_error()

### Try hushing various kinds of info

* html display
* print
* logging

In [15]:
%%hush
import pandas as pd
import logging
logging.getLogger().setLevel('DEBUG')
a = 1
for i in range(1000):
    print(i, end="\t")
    
logging.info("a lots of text, VERBOSITY!!"*100)

# a big dataframe
display(pd.DataFrame({"a":[1,2]*50}))

# a big output
pd.DataFrame({"b":range(100)})

<IPython.core.display.Javascript object>

### Capable of open/close verbosity during the main process

In [16]:
# Testing hush
%%hush
from time import sleep
for i in tqdm(range(10)):
    sleep(1)

UsageError: Line magic function `%%hush` not found.


### But we still want the error to be loud
> As such information should interrupt the user

In [13]:
# Testing hush when exception is raised
%%hush
def _test_hush():
    for i in range(20):
        print(f"some logging:\t{i}!")
    raise ValueError("but some error, because life!")



UsageError: Line magic function `%%hush` not found.


In [21]:
# hide
# To be able to run the tests in the Notebook
%pip install -q ipytest
import ipytest
import sys

sys.path.append(Path("..").resolve().absolute())
ipytest.autoconfig()


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Python37-32\python.exe -m pip install --upgrade pip' command.


In [ ]:
# exporti
from pathlib import Path
from shutil import copy

import pytest
from test.common.utils_4_tests import IMG_DIR, check_no_log, check_only_warning

It seems import above might not work... need to be explored.

In [ ]:
# exporti

images_rob = list((IMG_DIR / "robustness").glob("*.*"))

@pytest.mark.parametrize("img", images_rob, ids=[i.name for i in images_rob])
def test_check_img_error(img:Path, tmpdir, caplog):
    """Check check_img with incorrect images"""
    img_copy = tmpdir / img.name
    copy(img, img_copy)
    check_img(img_copy)
    check_only_warning(caplog, img.name)


def test_check_img_empty_wrong_suffix(tmpdir, caplog):
    """Check check_img with wrong image that does not have a correct extension"""
    img = Path(tmpdir) / "empty.txt"
    img.write_bytes(b"")
    check_img(img)
    check_no_log(caplog)

In [29]:
# hide
ipytest.run()

F                                                                        [100%]##vso[results.publish type=JUnit;runTitle='Pytest results';]e:\AnsysDev\_perso_repo\unpackai\nbs\test-output.xml
##vso[task.logissue type=error;]1 test(s) failed, 1 test(s) collected.
##vso[task.logissue type=warning;]Coverage XML was not created, skipping upload.

================================== FAILURES ===================================
_______________________________ test_check_img ________________________________

caplog = <_pytest.logging.LogCaptureFixture object at 0x059728B0>

    def test_check_img(caplog):
        """Dummy test of check_img"""
        img = Path("pic_0.bmp")
        img.write_bytes(b"0")
        check_img(img)
    
>       check_only_warning(caplog, "pic_0.bmp")
E       NameError: name 'check_only_warning' is not defined

C:\Users\jthuong\AppData\Local\Temp/ipykernel_18448/4147450626.py:7: NameError
------------------------------ Captured log call ------------------------------